# [Dogs vs. Cats Redux: Kernels Edition](http:////www.kaggle.com/competitions/dogs-vs-cats-redux-kernels-edition)

# Imports

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

from keras import layers
from keras import models, Model
from keras import optimizers
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras import applications

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import os

In [2]:
import zipfile

zip_files = ['test', 'train']

for zip_file in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

In [3]:
TRAIN_IMAGE_FOLDER_PATH="./train"
TRAIN_FILE_NAMES=os.listdir(TRAIN_IMAGE_FOLDER_PATH)

TEST_IMAGE_FOLDER_PATH="./test"
TEST_FILE_NAMES=os.listdir(TEST_IMAGE_FOLDER_PATH)

WIDTH=150
HEIGHT=150

In [4]:
targets=list()
train_full_paths=list()

for train_file_name in TRAIN_FILE_NAMES:
    target=train_file_name.split(".")[0]
    train_full_path=os.path.join(TRAIN_IMAGE_FOLDER_PATH, train_file_name)
    train_full_paths.append(train_full_path)
    targets.append(target)

train_dataset=pd.DataFrame()
train_dataset['train_image_path']=train_full_paths
train_dataset['target']=targets



test_full_paths=list()

for test_file_name in TEST_FILE_NAMES:
    test_full_path=os.path.join(TEST_IMAGE_FOLDER_PATH, test_file_name)
    test_full_paths.append(test_full_path)


test_dataset=pd.DataFrame()
test_dataset['test_image_path']=test_full_paths

In [5]:
target_counts=train_dataset['target'].value_counts()

target_counts

In [6]:
dataset_train, dataset_val=train_test_split(train_dataset, test_size=0.2, random_state=42)

In [7]:
train_datagen=ImageDataGenerator(rotation_range=15,
                                 rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1)

train_datagenerator=train_datagen.flow_from_dataframe(dataframe=dataset_train,
                                                      x_col="train_image_path",
                                                      y_col="target",
                                                      target_size=(WIDTH, HEIGHT),
                                                      class_mode="categorical",
                                                      batch_size=150)

In [8]:
val_datagen=ImageDataGenerator(rescale=1./255)

val_datagenerator=val_datagen.flow_from_dataframe(dataframe=dataset_val,
                                                   x_col="train_image_path",
                                                   y_col="target",
                                                   target_size=(WIDTH, HEIGHT),
                                                   class_mode="categorical",
                                                   batch_size=150)

In [9]:
test_dataset

In [10]:
test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_dataframe(test_dataset,
                                                  x_col='test_image_path',
                                                  y_col=None,
                                                  class_mode=None,
                                                  shuffle=False,
                                                  target_size=(WIDTH, HEIGHT))

# Implementing CNN Architecture with Keras

In [ ]:
model=models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape=(WIDTH, HEIGHT, 3)))
model.add(layers.Conv2D(32, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation="softmax"))

model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", 
             optimizer='adam',
             metrics=["accuracy"])

In [ ]:
modelHistory=model.fit(train_datagenerator,
                       epochs=1,
                       validation_data  = val_datagenerator,
                       validation_steps = dataset_val.shape[0]//150,
                       steps_per_epoch  = dataset_val.shape[0]//150)

# Transfer Learning 

In [17]:
model = tf.keras.applications.Xception(weights= "imagenet", include_top=False, input_shape=(WIDTH, HEIGHT, 3))

In [18]:
x = model.layers[-1].output
x = layers.Flatten()(x)
x = layers.Dense(256,activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation = 'softmax')(x)

model = Model(inputs = model.input, outputs = x)

In [19]:
for layer in model.layers[-10:]:
    layer.trainable = True
    
for layer in model.layers[:-10]:
    layer.trainable = False



model.compile(optimizer= 'adam'
             ,loss     = keras.losses.BinaryCrossentropy(from_logits=True)
             ,metrics  = keras.metrics.BinaryAccuracy())

In [20]:
dataset_val.shape[0]//150

In [21]:
model_xception_History=model.fit(train_datagenerator,
                                epochs=2,
                                validation_data =val_datagenerator,
                                validation_steps=dataset_val.shape[0]//150,
                                steps_per_epoch =dataset_val.shape[0]//150)

In [22]:
preds = model.predict(test_generator)

test_dataset['category'] = np.argmax(preds, axis=-1)

In [26]:
output = pd.DataFrame({'id': list(range(1, 12501)),
                       'label': test_dataset['category'] })

output.to_csv('submission2.csv', index=False)
print("Your submission was successfully saved!")